In [ ]:
import pandas as pd
import os
import json
import sys
import datetime
from yahoo_fin import stock_info as si
from dotenv import load_dotenv
import re

In [ ]:
load_dotenv()
# Specify the path to the desired directory
Workspace_directory_path = '/Workspace/Users/'
# Change the current working directory
os.chdir(Workspace_directory_path)
#Opening the stock market json file
with open(r'nasdaq-10-stocks.json','r') as stocks_json:
    stock_list = json.load(stocks_json)


In [ ]:
# Create an empty DataFrame to store the data
stock_info_agg = pd.DataFrame()

#Function for removing special characters from column name
def column_name_clean(stocks):
    cols = list(stocks.columns)
    new_col=[]
    for col in cols:
        new_cols = re.sub('[^a-zA-Z0-9 \n]', '', col)
        new_cols = new_cols.replace(' ','_')
        new_col.append(new_cols)
    stocks.columns = new_col
    return(stocks)

In [ ]:
# Function for transforming stock information
def stock_info_transform(stock_info,ticker,stock_name):
    stock_info = stock_info.transpose()
    new_header = stock_info.iloc[0]
    stock_info = stock_info[1:]
    stock_info.columns = new_header
    stock_info["ticker"] = ticker
    stock_info["stock"] = stock_name
    stock_info["Date"] = datetime.date.today()
    stock_info = column_name_clean(stock_info)
    return(stock_info)

#Function for retrieving stock info
def stock_info_load(stock_info_agg):
    for stock in stock_list:
        stock_info = si.get_stats_valuation(f"{stock['ticker']}")
        stock_info = stock_info_transform(stock_info,stock["ticker"],stock["stock"])
        if stock_info_agg.empty :
            stock_info_agg = stock_info
        else:
            stock_info_agg = pd.concat([stock_info_agg,stock_info],ignore_index=False)
    return(stock_info_agg)

In [ ]:
# Converting data frame to csv format
stock_info = stock_info_load(stock_info_agg)
stock_info = spark.createDataFrame(stock_info)

In [ ]:
temp_output_path = '/mnt/stocks_adlsgen2/nasdaq-stocks-info-ingest/temp_file/'
output_path = '/mnt/stocks_adlsgen2/nasdaq-stocks-info-ingest/stock_info.csv'
stock_info.repartition(1).write.mode("overwrite").option("header","true").csv(temp_output_path)
filenames = dbutils.fs.ls(temp_output_path)
for filename in filenames:
    if filename.name.endswith('.csv'):
        name = filename.name
dbutils.fs.cp(temp_output_path + name, output_path)
dbutils.fs.rm(temp_output_path,recurse=True)

True